In [13]:
import pandas as pd
import numpy as np

In [14]:
DiabetesTakingMed = pd.read_csv('DiabetesTakingMed.csv', index_col=0)

In [15]:
DiabetesTrain3 = DiabetesTakingMed[DiabetesTakingMed['IsTrain']==1].drop('IsTrain', axis=1)
DiabetesTrain3.index = list(range(len(DiabetesTrain3)))

DiabetesTest3 = DiabetesTakingMed[DiabetesTakingMed['IsTrain']==0].drop('IsTrain', axis=1)
DiabetesTest3.index = list(range(len(DiabetesTest3)))

trainX3 = DiabetesTrain3.drop('readmitted', axis=1)
trainY3 = DiabetesTrain3['readmitted'].replace([2, 1], [1, 0])

testX3 = DiabetesTest3.drop('readmitted', axis=1)
testY3 = DiabetesTest3['readmitted'].replace([2, 1], [1, 0])

In [16]:
#Import and run the model:
import xgboost
from xgboost.sklearn import XGBClassifier as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score as AUC

In [17]:
'''xgb = xgb()

params = {
        'n_estimators':[200, 400, 600, 800, 1000],
        'min_child_weight': [1, 2, 5, 10],
        'gamma': [0.1, 0.2, 0.5, 1, 1.5, 2, 5],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [2, 3, 5, 8]
        }

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=100, scoring='roc_auc', n_jobs=6, 
                                   cv=4, verbose=5, random_state=42)

random_search.fit(trainX2, trainY2)

random_search.best_params_'''

print("n_estimators=200, min_child_Weight=10, max_depth=5, gamma=5, colsample_bytree=0.6")

n_estimators=200, min_child_Weight=10, max_depth=5, gamma=5, colsample_bytree=0.6


In [18]:
from xgboost.sklearn import XGBClassifier as xgb

xgbF = xgb()
xgbF.set_params(n_estimators=200, min_child_weight=10, max_depth=5, gamma=5, colsample_bytree=0.6)

xgbF.fit(trainX3, trainY3)

predict = xgbF.predict(testX3)
predictprobs = xgbF.predict_proba(testX3)

#Make feature importances list:
AnyChangeXGBFeatures = pd.DataFrame({'Feature':pd.Series(trainX3.columns), 'Importance':xgbF.feature_importances_})

C:\Users\Dave\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [19]:
predictprobs

array([[0.92972445, 0.07027554],
       [0.9098509 , 0.09014908],
       [0.8505523 , 0.1494477 ],
       ...,
       [0.9491745 , 0.05082546],
       [0.9068813 , 0.09311873],
       [0.9207904 , 0.07920964]], dtype=float32)

In [20]:
#0.673 better than the 0.66 we had with other methods.
AUC(testY3, predictprobs[:,1])

0.6749736534379563

In [21]:
from sklearn.metrics import brier_score_loss as BSL

#0.0956 for XGBoost lower than the stack of logistic and RF (0.09589)
BSL(testY3, predictprobs[:,1])

0.09557876216512425

In [22]:
'''#Let's randomsearch for additional parameters:

xgbG = xgb()
xgbG.set_params(n_estimators=200, min_child_weight=10, max_depth=5, gamma=5, colsample_bytree=0.6)

params2 = {
        'learning_rate':[0.0001, 0.001, 0.01, 0.1, 0.2, 0.3],
        'max_delta_step': [0, 1, 2, 5, 8],
        'subsample':[0.5, 0.6, 0.7, 0.8, 0.9, 1]
        }

random_search2 = RandomizedSearchCV(xgbG, param_distributions=params2, n_iter=60, scoring='roc_auc', n_jobs=6, 
                                   cv=4, verbose=5, random_state=42)

random_search2.fit(trainX2, trainY2)

random_search2.best_params_'''

print("subsample=1, max_delta_step=5, learning_rate=0.1")

subsample=1, max_delta_step=5, learning_rate=0.1


In [24]:
xgbF2 = xgb()
xgbF2.set_params(n_estimators=200, min_child_weight=10, max_depth=5, gamma=5, colsample_bytree=0.6, max_delta_step=5)

xgbF2.fit(trainX3, trainY3)

predict = xgbF2.predict(testX3)
predictprobs = xgbF2.predict_proba(testX3)

#Make feature importances list:
AnyChangeXGBFeatures = pd.DataFrame({'Feature':pd.Series(trainX3.columns), 'Importance':xgbF2.feature_importances_})

C:\Users\Dave\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [26]:
#This is literally the same AUC score. It would seem that not a simgle tree had an altered prediction.
AUC(testY3, predictprobs[:,1])

0.6749736534379563

In [27]:
AnyChangeXGBFeatures.sort_values('Importance', ascending=False)
ChoiceColumns = list(AnyChangeXGBFeatures[AnyChangeXGBFeatures['Importance']>0]['Feature'])

In [28]:
trainX3C = trainX3[ChoiceColumns]
testX3C = testX3[ChoiceColumns]

xgbF2C = xgb()
xgbF2C.set_params(n_estimators=500, min_child_weight=10, max_depth=5, gamma=5, colsample_bytree=0.6, max_delta_step=5)

xgbF2C.fit(trainX3, trainY3)

predictC = xgbF2C.predict(testX3)
predictprobsC = xgbF2C.predict_proba(testX3)

C:\Users\Dave\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [29]:
#I also raised N_estimators as there is rarely a penalty for doing this, and it helped raise test AUC.
AUC(testY3, predictprobsC[:,1])

0.6753665800254492